In [2]:
%pip install numpy
%pip install tqdm
%pip install pandas
%pip install plotly.express

import numpy as np
from tqdm import tqdm
import plotly.express as px
import pandas as pd
from data.preprocessing import preprocess_csv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
preprocessed_dataset = preprocess_csv('data/data.csv', ['StockCode', 'InvoiceDate'])
preprocessed_dataset

,InvoiceNo,Description,Quantity,UnitPrice,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,35
1,536365,WHITE METAL LANTERN,6,3.39,17850.0,35
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,35
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,35
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,35
...,...,...,...,...,...,...
531264,581587,PACK OF 20 SPACEBOY NAPKINS,12,0.85,12680.0,13
531265,581587,CHILDREN'S APRON DOLLY GIRL,6,2.10,12680.0,13
531266,581587,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,12680.0,13
531267,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,12680.0,13


In [4]:
dataframe = preprocessed_dataset[['Quantity', 'UnitPrice']]
dataframe

,Quantity,UnitPrice
0,6,2.55
1,6,3.39
2,8,2.75
3,6,3.39
4,6,3.39
...,...,...
531264,12,0.85
531265,6,2.10
531266,4,4.15
531267,4,4.15


In [5]:
dataframe.drop_duplicates(inplace=True)
dataframe = dataframe[(dataframe < 3000).all(axis=1)]
dataframe = dataframe[(dataframe >= 0).all(axis=1)]

fig = px.scatter(dataframe, x='Quantity', y='UnitPrice')
fig.show()

/var/folders/b_/8zspp9vd6xq6tr1t99f7pgf80000gn/T/ipykernel_72547/4036559419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop_duplicates(inplace=True)


In [6]:
np_dataframe = dataframe.to_numpy()

In [7]:
class DBSCAN(object):

    def __init__(self, eps, minPts, dataset):
        self.eps = eps
        self.minPts = minPts
        self.dataset = dataset

    def fit(self):
        # Initialize cluster assignments to -1 (noise)
        cluster_idx = np.full(len(self.dataset), -1, dtype=int)
        
        # Track visited points
        visitedIndices = set()
        
        # Current cluster number
        C = 0
        
        # Iterate through all points
        for index in range(len(self.dataset)):
            # Skip if already visited
            if index in visitedIndices:
                continue
            visitedIndices.add(index)
            
            # Find neighbors
            neighborIndices = self.regionQuery(index)
            
            # Check density condition
            if len(neighborIndices) < self.minPts:
                # Noise point
                continue
            # Expand cluster
            self.expandCluster(index, neighborIndices, C, cluster_idx, visitedIndices)
            
            # Increment cluster number
            C += 1
        
        return cluster_idx

    def expandCluster(self, index, neighborIndices, C, cluster_idx, visitedIndices):
        """
        index: index of point P from self.dataset
        neighborIndices: np array -> indices of all points within eps distance from point P
        C: current cluster as an integer
        cluster_idx: np array -> current clusters for each point in the dataset
        visitedIndices: set of indices visited in dataset so far
        """
        # assign the current point to a cluster
        # Add the current point to the cluster
        cluster_idx[index] = C

        # Initialize seeds with current neighbors
        seeds = set(neighborIndices)

        seeds - visitedIndices
        # Process seeds until empty
        while seeds:
            current_seed = seeds.pop()
            
            # Skip if already visited
            if current_seed not in visitedIndices:
                visitedIndices.add(current_seed)
                cluster_idx[current_seed] = C
                
                # Find neighbors of current seed
                current_neighbors = self.regionQuery(current_seed)
                
                # If density condition met, expand cluster
                if len(current_neighbors) >= self.minPts:
                    # Add unvisited neighbors that are not already in a cluster
                    for neighbor in current_neighbors:
                        if neighbor not in visitedIndices:
                            seeds.add(neighbor)
                        
                        # Assign cluster if not already assigned
                        if cluster_idx[neighbor] == -1:
                            cluster_idx[neighbor] = C

    def regionQuery(self, pointIndex):
        """
        pointIndex: index of point P from self.dataset
        """
        distances = pairwise_dist(
            self.dataset[pointIndex].reshape(1, -1), 
            self.dataset
        )
        
        # find indices of points within eps distance
        indices = np.argwhere(distances[0] <= self.eps).flatten()
        return indices

def pairwise_dist(x, y):
    """
    x: N x D numpy array
    y: M x D numpy array
    """
    x_sq = np.sum(np.square(x), axis=1, keepdims=True) #nx1
    y_sq = np.sum(np.square(y), axis=1, keepdims=True).T #mx1
    xy = np.dot(x, y.T) #shape(n,m)
    return np.sqrt(np.maximum(x_sq + y_sq - 2 * xy, 0))

In [8]:
BEST_EPS = 10
BEST_POINTS = 3

dbscan = DBSCAN(BEST_EPS, BEST_POINTS, dataset=np_dataframe)
cluster_idx = dbscan.fit()
cluster_idx

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
colored_data = np.hstack((np_dataframe, cluster_idx.reshape(-1, 1)))

colored_data

array([[  6.  ,   2.55,   0.  ],
       [  6.  ,   3.39,   0.  ],
       [  8.  ,   2.75,   0.  ],
       ...,
       [ 66.  ,   0.38,   0.  ],
       [144.  ,   3.26,   0.  ],
       [ 84.  ,   0.79,   0.  ]])

In [10]:
colored_dataframe = pd.DataFrame(colored_data, columns=["Quantity", "UnitPrice", "Assignment"])

fig = px.scatter(colored_dataframe, x="Quantity", y="UnitPrice", color="Assignment")
fig.show()

In [11]:
colored_data_filtered = colored_data[colored_data[:, -1] != -1]

filtered_colored_dataframe = pd.DataFrame(colored_data_filtered, columns=["Quantity", "UnitPrice", "Assignment"])

fig = px.scatter(filtered_colored_dataframe, x="Quantity", y="UnitPrice", color="Assignment")
fig.show()

In [12]:
def compute_centers(points, cluster_idx):
    unique_clusters = np.unique(cluster_idx[cluster_idx >= 0])  # Ignore noise points
    centers = []
    for cluster in unique_clusters:
        cluster_points = points[cluster_idx == cluster]
        centers.append(np.mean(cluster_points, axis=0))
    return np.array(centers)

def calculate_silhouette_score(points, assignments, centers):
    n_points = points.shape[0]
    silhouette_scores = np.zeros(n_points)
    for i in range(n_points):
        current_cluster = assignments[i]
        same_cluster_points = points[assignments == current_cluster]
        if len(same_cluster_points) > 1:
            a_i = np.mean(np.linalg.norm(same_cluster_points - points[i], axis=1))
        else:
            a_i = 0
        b_i = float('inf')
        for k in range(centers.shape[0]):
            if k != current_cluster:
                other_cluster_points = points[assignments == k]
                if other_cluster_points.size > 0:
                    avg_dist_to_other_cluster = np.mean(np.linalg.norm(other_cluster_points - points[i], axis=1))
                    b_i = min(b_i, avg_dist_to_other_cluster)
        if max(a_i, b_i) > 0:
            silhouette_scores[i] = (b_i - a_i) / max(a_i, b_i)
        else:
            silhouette_scores[i] = 0
    return np.mean(silhouette_scores)
silhouette_score = calculate_silhouette_score(np_dataframe, cluster_idx, compute_centers(np_dataframe, cluster_idx))

silhouette_score

0.6326280362414816